# Borrower reliability research
## 1. Data overview

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('D:\Jupyter\\6. Data preprocessing\data_prep.csv')
print(data.head(20))

    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0    -926.185831         27               высшее             0   
6          0   -2879.202052         43               высшее             0   
7          0    -152.779569         50              СРЕДНЕЕ             1   
8          2   -6929.865299         35               ВЫСШЕЕ             0   
9          0   -2188.756445         41              среднее             1   
10         2   -4171.483647         36               высшее             0   
11         0    -792.701887         40              среднее             1   

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## 2. Data Preprocessing
### 2.1. Missing values

In [4]:
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

'total_income' is most dependant on 'income_type', thus we will insert median values in missing data for every type seperatly

In [5]:
for type in data['income_type'].unique():
    type_income = data[data['income_type'] == type]['total_income'].median()
    data.loc[(data['income_type'] == type) & (data['total_income'].isna()), 'total_income'] = type_income

### 2.2. Anomalious data

In [6]:
data['days_employed'] = data['days_employed'].abs()
data.groupby('income_type')['days_employed'].median()

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

In [7]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [8]:
data = data.drop(data[data['children'].isin([-1,20])].index)
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### 2.3. Missing values, continuing
For 'days_employed' it's the same as 'total_income'

In [9]:
for type in data['income_type'].unique():
    data.loc[(data['income_type'] == type) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[data['income_type'] == type,'days_employed'].median()
data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### 2.4. Changing Data Types

In [10]:
data['total_income'] = data['total_income'].astype('int')

### 2.5. Duplicate processing

In [11]:
data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [12]:
data.duplicated().sum()

71

### 2.6. Data categorisation

The following categories for 'total_income' are used:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [13]:
def categorize_income(income):
    if income<30001:
        return 'E'
    elif income<50001:
        return 'D'
    elif income<200001:
        return 'C'
    elif income<1000001:
        return 'B'
    else:
        return 'A'
    
data['total_income_category'] = data['total_income'].apply(categorize_income)
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

The following categories for 'purpose' are used:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [14]:
def categorize_purpose(purpose):
    if 'автомоб' in purpose:
        return 'операции с автомобилем'
    elif any(x in purpose for x in ['недвиж','жиль','кварт']):
        return 'операции с недвижимостью'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'образован' in purpose:
        return 'получение образования'
    else:
        return 'неопределено'
    
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## 3. Data exploration
### 3.1. Relationship between the number of children and repayment of the loan on time
Loan repayment on time is given as 1 or 0 in the debt column, so you can check the dependence by dividing the sum of the searched debt groups by the group size

In [15]:
data_pic = data.pivot_table(index = 'children', values = 'debt', aggfunc = ['count', 'sum', 'mean'])\
    .sort_values(by = ('children'))\
    .style.format({('mean', 'debt') : '{:.2%}'})
data_pic

,count,sum,mean
,debt,debt,debt
children,,,
0,14149,1063,7.51%
1,4818,444,9.22%
2,2055,194,9.44%
3,330,27,8.18%
4,41,4,9.76%
5,9,0,0.00%


**Conclusion:**

The group without children has a reduced proportion of debtors; no other dependencies are observed. A group of 5 children is too small to draw conclusions.

In [16]:
data_sum = data.groupby('children')['debt'].sum()
data_count = data.groupby('children')['debt'].count()
data_div = data_sum / data_count
data_div_child = (data_sum[1:6].sum() / data_count[1:6].sum())
(data_div_child - data_div[0]) / data_div_child

0.18548501687305965

On average, there are 19% fewer debtors in the group without children
### 3.2. Relationship between marital status and loan repayment on time

In [18]:
display(data['family_status'].unique())

data.loc[data['family_status'] == 'гражданский брак','family_status'] = 'женат / замужем'
data.loc[data['family_status'] == 'в разводе','family_status'] = 'вдовец / вдова'

data_sum = data.groupby('family_status')['debt'].sum()

data_count = data.groupby('family_status')['debt'].count()
data_div = data_sum / data_count
print('Share of debtors')
data_div.sort_values(ascending = False)

array(['женат / замужем', 'вдовец / вдова', 'Не женат / не замужем'],
      dtype=object)

Share of debtors


family_status
Не женат / не замужем    0.097535
женат / замужем          0.079699
вдовец / вдова           0.068660
Name: debt, dtype: float64

**Conclusion:**

Single and unmarried have the largest share of debtors, previously married have the smallest, and currently married have an average share.
### 3.3. Relationship between income level and loan repayment on time

In [20]:
data.pivot_table(index = 'total_income_category', values = 'debt', aggfunc = ['count', 'sum', 'mean'])\
    .sort_values(by = ('count', 'debt'), ascending = False)\
    .style.format({('mean', 'debt') : '{:.2%}'})

,count,sum,mean
,debt,debt,debt
total_income_category,,,
C,15992,1353,8.46%
B,5014,354,7.06%
D,349,21,6.02%
A,25,2,8.00%
E,22,2,9.09%


**Conclusion:**

We can safely discard groups A and E as statistically insignificant. The largest group, medium-earning Group C, had the highest proportion of debtors, higher than high-earning Group B. The smallest share of debtors is in group D with low earnings.

### 3.4. Relationship between purposes of a loan and loan repayment on time

In [21]:
data_sum = data.groupby('purpose_category')['debt'].sum()

data_count = data.groupby('purpose_category')['debt'].count()
data_div = data_sum / data_count
print('Share of debtors')
data_div.sort_values(ascending = False)

Share of debtors


purpose_category
операции с автомобилем      0.093284
получение образования       0.092319
проведение свадьбы          0.078306
операции с недвижимостью    0.072356
Name: debt, dtype: float64

**Conclusion:**

2 groups of similar shares: car and education have a larger share of debtors than wedding and real estate.

In [22]:
data_sum_purpose1 = data_sum['операции с автомобилем'] + data_sum['получение образования']
data_count_purpose1 = data_count['операции с автомобилем'] + data_count['получение образования']
data_div_purpose1 = data_sum_purpose1 / data_count_purpose1
data_sum_purpose2 = data_sum['проведение свадьбы'] + data_sum['операции с недвижимостью']
data_count_purpose2 = data_count['проведение свадьбы'] + data_count['операции с недвижимостью']
data_div_purpose2 = data_sum_purpose2 / data_count_purpose2
(data_div_purpose1 - data_div_purpose2)/data_div_purpose1

0.20903377058707306

On average, there are 21% fewer debtors in the group with weddings and real estate

**Overall:**

Data shows that having children leads to an increased chance of getting into loan debt. At the same time, those who are not married also have an increased chance of incurring debt. It makes sense to conduct a joint analysis of the presence of children and marital status in the future. In terms of earnings: group C with average earnings has the highest proportion of debtors. By purpose: loans for cars and education turned out to have the largest share of debtors.